In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import xlrd

In [3]:
year = 2009

In [4]:
directory_2009 = './'+str(year)+'Data.xls'

In [5]:
overallDF = pd.DataFrame()
book = xlrd.open_workbook(directory) 
num_sheets = len(book.sheets())

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [6]:
num_sheets

59

In [7]:
sheet_names = book.sheet_names()
sheet_names

[u'Alex Miller - 02-ACIR',
 u'Alex Miller - 05-Patient Satisf',
 u'Alex Miller - 01-Patient-Physic',
 u'Alex Miller - Patient Note Scor',
 u'Alex Miller - 04-Physical Exam ',
 u'Alex Miller - 03-History Alex M',
 u'Carol Whitman - 04-Patient Sati',
 u'Carol Whitman - 01-Patient-Phys',
 u'Carol Whitman - 03-SPIKES Proto',
 u'Carol Whitman - 02-History Caro',
 u'Corey Wolfe - 05-Patient Satisf',
 u'Corey Wolfe - 01-Patient-Physic',
 u'Corey Wolfe - 02-ACIR',
 u'Corey Wolfe - Patient Note Scor',
 u'Corey Wolfe - 04-Physical Exam ',
 u'Corey Wolfe - 03-History Corey ',
 u'Dana Mitchell - 05-Patient Sati',
 u'Dana Mitchell - 01-Patient-Phys',
 u'Dana Mitchell - 02-ACIR',
 u'Dana Mitchell - Patient Note Sc',
 u'Dana Mitchell - 04-Physical Exa',
 u'Dana Mitchell - 03-History Dana',
 u'Jamie Browning - 05-Patient Sat',
 u'Jamie Browning - 01-Patient-Phy',
 u'Jamie Browning - 02-ACIR',
 u'Jamie Browning - Patient Note S',
 u'Jamie Browning - 03-History Jam',
 u'Jamie Browning - 04-Physical Ex',

In [8]:
for i in range(num_sheets):
    this_sheet = sheet_names[i]
    df = pd.read_excel(directory, this_sheet)
    overallDF = pd.concat([overallDF, df], axis=1)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

In [9]:
overallDF.shape

(78, 971)

In [10]:
overallDF = overallDF.drop([0,1,2])

In [11]:
overallDF.head()

,Student Name_SP_TW,School ID_SP_TW,Class Year_SP_TW,Patient Name_SP_TW,Evaluator Name_SP_TW,Exam Date/Time_SP_TW,Q1_A_SP_TW,Q2_A_SP_TW,Q3_A_SP_TW,Q4_A_SP_TW,...,Q10_Hx_SP_LAP,Q11_Hx_SP_LAP,Q12_Hx_SP_LAP,Q13_Hx_SP_LAP,Q14_Hx_SP_LAP,Q15_Hx_SP_LAP,Total_Hx_SP_LAP,Total_SP_LAP,Question Raw Data,Unnamed: 1
3,"Angarita, Benjamin",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-12 11:00:04.420000,3,3,3,2,...,4,4,4,4,4,0,56.0,56.0,Students,75
4,"Austin, Naola",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-11 12:22:50.763000,4,4,3,3,...,0,4,4,4,4,4,48.0,48.0,NaN,NaN
5,"Ayodele, Maranatha",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-22 13:26:49.560000,3,3,4,3,...,0,0,0,0,4,0,32.0,32.0,Cases,Alex Miller
6,"Batavia, Ashita",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-16 09:56:40.403000,3,3,3,3,...,0,0,0,0,4,0,36.0,36.0,NaN,Carol Whitman
7,"Beesley, Sarah",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-24 12:06:31.140000,3,4,3,2,...,0,0,0,0,4,0,36.0,36.0,NaN,Corey Wolfe


In [12]:
for i in overallDF.columns:
    print (i)

Student Name_SP_TW
School ID_SP_TW
Class Year_SP_TW
Patient Name_SP_TW
Evaluator Name_SP_TW
Exam Date/Time_SP_TW
Q1_A_SP_TW
Q2_A_SP_TW
Q3_A_SP_TW
Q4_A_SP_TW
Q5_A_SP_TW
Q6_A_SP_TW
Q7_A_SP_TW
Q8_A_SP_TW
Q9_A_SP_TW
Q10_A_SP_TW
Total_A_SP_TW
Total_SP_TW
Student Name_SP_TW
School ID_SP_TW
Class Year_SP_TW
Patient Name_SP_TW
Evaluator Name_SP_TW
Exam Date/Time_SP_TW
Q1_PS_SP_TW
Q2_PS_SP_TW
Total_PS_SP_TW
Total_SP_TW
Student Name_SP_TW
School ID_SP_TW
Class Year_SP_TW
Patient Name_SP_TW
Evaluator Name_SP_TW
Exam Date/Time_SP_TW
Q1_PPI_SP_TW
Q2_PPI_SP_TW
Q3_PPI_SP_TW
Q4_PPI_SP_TW
Q5_PPI_SP_TW
Q6_PPI_SP_TW
Q7_PPI_SP_TW
Q8_PPI_SP_TW
Q9_PPI_SP_TW
Q10_PPI_SP_TW
Q11_PPI_SP_TW
Q12_PPI_SP_TW
Q13_PPI_SP_TW
Q14_PPI_SP_TW
Q15_PPI_SP_TW
Q16_PPI_SP_TW
Total_PPI_SP_TW
Total_SP_TW
Student Name_Ad_TW
School ID_Ad_TW
Class Year_Ad_TW
Patient Name_Ad_TW
Evaluator Name_Ad_TW
Exam Date/Time_Ad_TW
Q1_DR_Ad_TW
Total_DR_Ad_TW
Total_Ad_TW
Student Name_SP_TW
School ID_SP_TW
Class Year_SP_TW
Patient Name_SP_TW
Evaluat

#### Identify pattern of columns for each subdivision.  Then find row-wise mean and variance for those selected columns and add to end of DF

In [13]:
ppi_initiation_qs = ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"]
ppi_info_gather = ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"]
ppi_closing = ["Q13_PPI", "Q14_PPI", "Q15_PPI"]




In [14]:
def add_mean_and_var_column(question_names, new_col_name):
    selected_cols = list(overallDF.loc[:, overallDF.columns.str.contains(('|'.join(question_names)))].columns)
    overallDF["mean_" + new_col_name] = overallDF[selected_cols].mean(axis=1)
    overallDF["var_" + new_col_name] = overallDF[selected_cols].var(axis=1)

add_mean_and_var_column(ppi_initiation_qs, 'ppi_initiation')
add_mean_and_var_column(ppi_info_gather, 'ppi_info_gather')
add_mean_and_var_column(ppi_closing, 'ppi_closing')

In [15]:
overallDF.head()

,Student Name_SP_TW,School ID_SP_TW,Class Year_SP_TW,Patient Name_SP_TW,Evaluator Name_SP_TW,Exam Date/Time_SP_TW,Q1_A_SP_TW,Q2_A_SP_TW,Q3_A_SP_TW,Q4_A_SP_TW,...,Total_Hx_SP_LAP,Total_SP_LAP,Question Raw Data,Unnamed: 1,mean_ppi_initiation,var_ppi_initiation,mean_ppi_info_gather,var_ppi_info_gather,mean_ppi_closing,var_ppi_closing
3,"Angarita, Benjamin",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-12 11:00:04.420000,3,3,3,2,...,56.0,56.0,Students,75,2.769231,3.475113,2.807692,3.380134,2.266667,4.064368
4,"Austin, Naola",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-11 12:22:50.763000,4,4,3,3,...,48.0,48.0,NaN,NaN,3.923077,0.307692,3.692308,1.147125,3.333333,2.298851
5,"Ayodele, Maranatha",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-22 13:26:49.560000,3,3,4,3,...,32.0,32.0,Cases,Alex Miller,3.692308,1.158371,3.692308,1.147125,2.933333,3.236782
6,"Batavia, Ashita",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-16 09:56:40.403000,3,3,3,3,...,36.0,36.0,NaN,Carol Whitman,3.384615,2.123680,3.615385,1.404033,3.466667,1.912644
7,"Beesley, Sarah",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-24 12:06:31.140000,3,4,3,2,...,36.0,36.0,NaN,Corey Wolfe,3.846154,0.603318,3.692308,1.147125,2.800000,3.475862


In [16]:
hx_physical =["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"]
hx_social = ["Q9_Hx"]

add_mean_and_var_column(hx_physical, 'hx_physical')
add_mean_and_var_column(hx_social, 'hx_social')

In [17]:
pe_handwash = ["Q1_PE"]
pe_phys_check = ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"]
pe_modesty = ["Q3_PE"]

add_mean_and_var_column(pe_handwash, 'pe_handwash')
add_mean_and_var_column(pe_phys_check, 'pe_phys_check')
add_mean_and_var_column(pe_modesty, 'pe_modesty')

# 2008

In [104]:
directory_2008 = './2008Data.xls'

overallDF_2008 = pd.DataFrame()
book_2008 = xlrd.open_workbook(directory_2008) 
num_sheets_2008 = len(book_2008.sheets())

sheet_names_2008 = book_2008.sheet_names()
sheet_names_2008

#GET ALL STUDENTS TO USE AS THE INDEX
book_students_2008 = xlrd.open_workbook('./2008Students.xlsx')
all_students_2008 = pd.read_excel('2008Students.xlsx', 'Sheet1', header=None).iloc[:,0].tolist()


In [105]:
for i in range(num_sheets_2008):
    this_sheet = sheet_names_2008[i]
    df = pd.read_excel(directory_2008, this_sheet)
    df.set_index(df.columns[0], inplace=True)
    df = df.reindex(all_students_2008)
    df = df.reset_index()
    overallDF_2008 = pd.concat([overallDF_2008, df], axis=1)



In [106]:
overallDF_2008

,Student Name_Ad_TW,School ID_Ad_TW,Class Year_Ad_TW,Patient Name_Ad_TW,Evaluator Name_Ad_TW,Exam Date/Time_Ad_TW,Q1_DR_Ad_TW,Total_DR_Ad_TW,Total_Ad_TW,Student Name_SP_TW,...,Q7_Hx_SP_LAP,Q8_Hx_SP_LAP,Q9_Hx_SP_LAP,Q10_Hx_SP_LAP,Q11_Hx_SP_LAP,Q12_Hx_SP_LAP,Q13_Hx_SP_LAP,Q14_Hx_SP_LAP,Total_Hx_SP_LAP,Total_SP_LAP
0,"Adler, Karen",NaN,2008.0,"Lieth, Douglas","Slade, Michael",2008-02-26 14:10:14.607000,10,10.0,10.0,"Adler, Karen",...,4,4,4,4,4,4,4,4,56.0,56.0
1,"Ala, Owen",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-07-26 13:39:22.607000,7.5,7.5,7.5,"Ala, Owen",...,4,0,4,0,4,4,0,4,44.0,44.0
2,"Bauman, Jessica",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-07-24 14:17:59.450000,10,10.0,10.0,"Bauman, Jessica",...,4,0,4,0,0,4,4,4,44.0,44.0
3,"Blumenthal, Jill",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-07-19 12:58:09.873000,7.5,7.5,7.5,"Blumenthal, Jill",...,4,0,4,0,4,4,0,4,44.0,44.0
4,"Bomsztyk, Elan",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-08-07 13:09:17.340000,7.5,7.5,7.5,"Bomsztyk, Elan",...,4,4,4,0,0,0,0,4,36.0,36.0
5,"Bradshaw, Chanda",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-07-23 10:22:39.263000,7.5,7.5,7.5,"Bradshaw, Chanda",...,0,4,4,4,4,4,0,4,48.0,48.0
6,"Brooks, Gabriel",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-07-23 09:00:29.950000,10,10.0,10.0,"Brooks, Gabriel",...,0,0,4,4,4,0,0,4,40.0,40.0
7,"Burton, Rebecca",NaN,2008.0,"Lieth, Douglas","Slade, Michael",2007-07-18 09:01:02.750000,5,5.0,5.0,"Burton, Rebecca",...,4,4,4,4,4,4,4,4,56.0,56.0
8,"Cannon, Donald",NaN,2008.0,"Levine, Neil","Slade, Michael",2007-08-07 09:49:27.077000,7.5,7.5,7.5,"Cannon, Donald",...,0,0,4,4,4,0,0,4,40.0,40.0
9,"Casemyr, Natalie",NaN,2008.0,"Lieth, Douglas","Slade, Michael",2007-07-18 12:45:17.183000,10,10.0,10.0,"Casemyr, Natalie",...,4,4,4,4,4,4,4,4,52.0,52.0


[u'Adler, Karen',
 u'Ala, Owen',
 u'Bauman, Jessica',
 u'Blumenthal, Jill',
 u'Bomsztyk, Elan',
 u'Bradshaw, Chanda',
 u'Brooks, Gabriel',
 u'Burton, Rebecca',
 u'Cannon, Donald',
 u'Casemyr, Natalie',
 u'Chen, Connie',
 u'Chevalier, Michelle',
 u'Choi, Jason',
 u'Chu, David',
 u'Cimino, Ashley',
 u'Colantino, Alison',
 u'Collins, Sean',
 u'Connors, Nicholas',
 u'Costa, Ana',
 u'Daines, William',
 u'Davis, Adrienne',
 u'Deobhakta, Avnish',
 u'Deshpande, Ohm',
 u'Diamond, Elan',
 u'Doyle, Lucy',
 u'Enschede, Elizabeth',
 u'Epstein, Jason',
 u'Ford, Michael',
 u'Fraser, Claire',
 u'Gade, Terence',
 u'Gardes, Jonathan',
 u'Genisca, Alicia',
 u'Gianotti , Robert',
 u'Goodman, Linda',
 u'Graustein, Andrew',
 u'Greninger, Daniel',
 u'Gupta, Gaorav',
 u'Hamilton, Tanya',
 u'Harder, Jessica',
 u'Harding, Elizabeth',
 u'Hellawell, Jennifer',
 u'Hinson, Akili',
 u'Hochsztein, Ariella',
 u'Horning, Matthew',
 u'Howard, Marion',
 u'Hudson, Andrew',
 u'Hung, Kenneth',
 u'Imahiyerobo, Joyce',
 u'Inr